# Polecenie

Polecenie (ang. Command) to czynnościowy wzorzec projektowy, który enkapsuluje żądanie jako obiekt, pozwalając na jego parametryzację, kolejkowanie, logowanie oraz późniejsze wykonanie. Wzorzec polecenie oddziela nadawcę polecenia od jego odbiorcy, co umożliwia dynamiczne zarządzanie działaniami, np. poprzez implementację operacji cofania (undo) lub wykonywania zadań w kolejności. Wzorzec ten składa się z czterech kluczowych elementów: polecenia, które określa akcję do wykonania; odbiorcy, który wykonuje akcję; wywołującego, który inicjuje polecenie oraz klienta, który konfiguruje polecenie.

## Przeznaczenie i zastosowanie

- Oddzielenie obiektów wysyłających żądania od obiektów, które je wykonują.
- Możliwość kolejkowania, planowania i logowania poleceń.
- Umożliwienie dynamicznej konfiguracji i komponowania poleceń.

<img src="img/Command_Design_Pattern_UML.jpg">

<img src="img/Command_pattern.svg" width="45%">

## Implementacja w postaci łańcucha komend

Klasa odbiornika odpowiedzialnego za wykonywanie poleceń

In [ ]:
class Receiver:
    @staticmethod
    def execute(data: str = ""):
        print(f"An action executed with data: {data}")

Klasa reprezentująca polecenie

In [ ]:
class Command:
    _receiver: Receiver

    def __init__(self, receiver: Receiver) -> None:
        self._receiver = receiver

    def execute(self, data: str = "") -> None:
        print("The command passed to receiver.")
        self._receiver.execute(data)

    def __str__(self) -> str:
        return f"Command to receiver {self._receiver}"

Klasa wywołującego. Oprócz kolejkowania komend do wykoanania na odbiornikach, zawiera także ich pełną historię wywołania.

In [ ]:
class Caller:
    _commands: list

    def __init__(self) -> None:
        self._commands = []

    def store_command(self, command: Command, data: str = "") -> None:
        self._commands.append((command, data))

    def execute_commands(self) -> None:
        for command, data in self._commands:
            command.execute(data)

    def show_commands(self) -> None:
        for command, data in self._commands:
            print(command)

    def show_history(self) -> None:
        for command, data in self._commands[::-1]:
            print(command)

Kod klienta

In [ ]:
tool = Receiver()
cmd1 = Command(tool)
cmd2 = Command(tool)
caller = Caller()

In [ ]:
caller.store_command(cmd1, "do command 1")
caller.store_command(cmd2, "do command 2")
caller.execute_commands()

In [ ]:
caller.show_history()

## Implementacja na przykładzie przełącznika latarki

In [ ]:
from abc import ABC, abstractmethod
from collections import deque

Przełącznik - klasa wywołująca

In [ ]:
class Switch:
    history: deque

    def __init__(self) -> None:
        self.history = deque()

    def execute(self, command):
        self.history.appendleft(command)
        command.execute()

    def get_history(self):
        return self.history

Latarka - klasa reprezentująca odbiornik

In [ ]:
class Torch:
    state: str

    def __init__(self, state: str = "off") -> None:
        self.state = state

    def turn_on(self) -> None:
        print('Torch turned on')
        self.state = 'on'

    def turn_off(self) -> None:
        print('Torch turned off')
        self.state = 'off'

Klasa reprezentująca abstrakcję polecenia

In [ ]:
class Command(ABC):
    torch: Torch

    def __init__(self, torch: Torch) -> None:
        self.torch = torch

    @abstractmethod
    def execute(self) -> None:
        pass

Implementacje klas poleceń

In [ ]:
class TurnLightOn(Command):
    def execute(self) -> None:
        self.torch.turn_on()

In [ ]:
class TurnLightOff(Command):
    def execute(self) -> None:
        self.torch.turn_off()

Implementacja klasy wywoływacza

In [ ]:
class TorchSwitcher:
    torch: Torch
    switch: Switch

    def __init__(self, torch: Torch, switch: Switch) -> None:
        self._torch = torch
        self._switch = switch

    def toggle(self, cmd: str) -> None:
        if cmd.lower() == "on":
            self._switch.execute(TurnLightOn(self._torch))
        else:
            self._switch.execute(TurnLightOff(self._torch))

Kod klienta

In [ ]:
my_torch = Torch()
my_switcher = Switch()
my_torch_switcher = TorchSwitcher(my_torch, my_switcher)

In [ ]:
command = 'on'
my_torch_switcher.toggle(command)

In [ ]:
command = 'off'
my_torch_switcher.toggle(command)

In [ ]:
print(my_switcher.get_history())

## Podsumowanie

Polecenie to czynnościowy wzorzec projektowy, który enkapsuluje żądanie jako obiekt, pozwalając na jego parametryzację, kolejkowanie, logowanie oraz późniejsze wykonanie. Takie podejście rodzi konsekwencje:
- dana komenda jest uruchamiana na żądanie,
- czynność wykonywana na obiekcie jest w postaci odseparowanej (jako osobny obiekt),
- obiekt wywołujący nie wie w jaki sposób działa ostateczny obiekt,
- polecenia można uruchamiać zarówno sekwencyjnie, jak i równolegle.